In [1]:
from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt

import dash
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd
import os
import numpy as np
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import shapefile
import descartes
from shapely.geometry import Point
import gmaps
import gmaps.datasets
import geopandas
import plotly.graph_objects as go # or plotly.express as px
import plotly.express as px
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.figure_factory as ff
import re
from datetime import datetime as dt
import json
import dash_bootstrap_components as dbc

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     counties = json.load(response)

In [2]:
merged = pd.read_csv('/Users/jiaqilu/Desktop/ENV19/methane_2010_20200713.csv')
merged['CH4CDSup_A'] = merged['CH4CDSup_A'] * (1e6)

In [ ]:

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}



#Side Panel  ---------------------------------------------------------------

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 900,
    "bottom": 0,
    "width": "60rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}


sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        html.P(
            "A simple sidebar layout with statistical graphs", className="lead"
        ),
        
        dcc.Graph(
        id='stats-graph'
        ),
    
    ],
    style=SIDEBAR_STYLE,
    
    
)

content_sidebar = html.Div(id="page-content", style=CONTENT_STYLE)


#Dropdown menu ---------------------------------------------------------------

# separate station variables and county variables
# this list is not complete
variable_type = {
    'XCO2':'county',
    'XCH4':'county',
    'NO2':'station',
    'PM2.5':'county',
    'Dissolved Oxygen':'station',
    'Orthophosphate':'station',
    'COVID Cases':'county'
}
# unit for each variable
unitmap = {
    'XCO2':'ppm of ',
    'XCH4':'ppm of ',
    'NO2':'ppm of ',
    'PM2.5':'ppm of ', 
    'Dissolved Oxygen':'blah unit of ',
    'Orthophosphate':'blah unit of ',
    'COVID Cases':''
}

dropdown = html.Div([
    dcc.Dropdown(id="slct_var",
                 options=[
                     {"label": "Greenhouse Gas - XCO2", "value": "XCO2"},
                     {"label": "Greenhouse Gas - XCH4", "value": "XCH4"},
                     {"label": "Air Quality - NO2", "value": "NO2"},
                     {"label": "Air Quality - PM2.5", "value": "PM2.5"},
                     {"label": "Water Quality - Dissolved Oxygen", "value": "Dissolved Oxygen"},
                     {"label": "Water Quality - Orthophosphate", "value": "Orthophosphate"},
                     {"label": "COVID Cases", "value": "COVID Cases"}],
                 multi=False,
                 value="XCO2",
                 style={'width': "40%"}
                 )
])

content_dropdown = html.Div(id="dropdown-content")

#APP LAYOUT---------------------------------------------------------------
app.layout = html.Div([
    html.H1("Texas Environmental Data and COVID19", style={'text-align': 'center'}),
    
    html.Div([dcc.Location(id="sel_var"), dropdown, content_dropdown]),
    
    html.Div([
        dcc.Graph(id='the_graph')
    ]),
    
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(2015, 1, 1),
        max_date_allowed=dt(2020, 7, 1),
        initial_visible_month=dt(2020, 4, 1),
        date=str(dt(2020, 6, 1))
    ),
    html.Div(id='output-container-date-picker-single'),
    
    html.Div([
            dcc.Markdown("""
                **Click region**
                Click on regions on the graph.
            """),
            html.Pre(id='click-data', style=styles['pre']),
        ], className='three columns'),
    
    html.Div([dcc.Location(id="url"), sidebar, content_sidebar]),
    
    
    
])



#---------------------------------------------------------------


@app.callback(
    [Output('output-container-date-picker-single', 'children'),
    Output(component_id='the_graph', component_property='figure')
    ],
    [Input('my-date-picker-single', 'date')])

def update_output(date):
    string_prefix = 'You have selected: '
    
    scale = merged['CH4CDSup_A'].tolist()
    endpts = list(np.mgrid[min(scale):max(scale):16j])
    date_string = '2020-05-01'
    
    if date is not None:
        date = dt.strptime(re.split('T| ', date)[0], '%Y-%m-%d')
        date_string = date.strftime('%Y-%m-%d')
        temp = merged.loc[merged.Time == date_string,:]

        trace= go.Choroplethmapbox(geojson = counties,
                           locations = temp['fips'],
                           z = temp['CH4CDSup_A'],
                           colorscale='hot',
                           reversescale = True,
                           colorbar_thickness=20,
                           text = temp['CNTY_NM'],
                           
                           marker_line_color='white',
                           customdata = temp['fips'],
                           hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                             '<br> <b>Val </b>: %{z}<br>'+ '<extra></extra>' 
                        )
        fig= go.Figure(data = trace)
        mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'

        fig.update_layout(title_text= 'Texas Heatmap',
                          title_x=0.5, width = 700, height=700,
                          mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                         accesstoken= mapboxt,style = 'basic',
                                         zoom=4.5,
                                       ),
                         clickmode='event+select'
                         );

        
        return (string_prefix + date_string, fig)

    
#Update side panel---------------------------------------------------------------    
@app.callback(
    [Output('click-data', 'children'),
     Output(component_id = 'stats-graph', component_property='figure')],
    [Input('the_graph', 'clickData')])
def display_click_data(clickData):
    string = json.dumps(clickData)
    dictionary = json.loads(string)
    if dictionary and 'points' in dictionary.keys():
        point = dictionary['points'][0]
        fips = int(point['location'])
        select =  merged.loc[merged.fips == fips,:]
        select = select.sort_values(by=['Time'])
        select['Time'] = pd.to_datetime(select['Time'])
        county = str(select['CNTY_NM'].iloc[1])
        print(county)
#         county_name = select['CNTY_NM'][0]
        fig = px.line(select, x="Time", y="CH4CDSup_A", title='Concentration of CH4 at '+ county +' county')
        
        fig.update_layout(xaxis_title='Time',
                   yaxis_title='CH4 Concentration (*e^-6 ppv)')
        
        fig.update_traces(marker_size=20)
        fig.update_xaxes(
            rangeslider_visible=True,
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            )
        )
        
        return (fips,fig)
    else:
        raise Exception("POST /_dash-update-component HTTP/1.1")
        

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', port=5000)

Dash is running on http://0.0.0.0:5000/

Dash is running on http://0.0.0.0:5000/

Dash is running on http://0.0.0.0:5000/

Dash is running on http://0.0.0.0:5000/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jul/2020 11:52:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2020 11:52:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2020 11:52:44] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/jiaqilu/

127.0.0.1 - - [19/Jul/2020 11:52:46] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Jul/2020 11:52:48] "POST /_dash-update-component HTTP/1.1" 200 -
